In [ ]:
!pip install --upgrade pandas
!pip install --upgrade torch
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: conda: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-xi33hg0y
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-xi33hg0y
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done


In [ ]:
import random
import cv2
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.transforms import InterpolationMode
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn import preprocessing
import torchvision.models as models
from sklearn.metrics import f1_score
from tqdm.auto import tqdm
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import seaborn as sns
import clip
import nltk
from nltk.tokenize import word_tokenize


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(69)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# %cd /content/gdrive/MyDrive/wall

In [ ]:
# !unzip "open.zip"
# %cd /content

In [ ]:
CFG = {
    'IMG_SIZE':256,#b3 300 b0 = 224
    'EPOCHS':30,
    'LEARNING_RATE':1e-04,#  7.8125e-07, 6.25e-06
    'BATCH_SIZE':64,
    'SEED':69
}

In [ ]:
path = '/content/gdrive/MyDrive/wall'
model_path = f"{path}/model_{CFG['BATCH_SIZE']}_{CFG['LEARNING_RATE']}.pth"
test = pd.read_csv(f'{path}/test.csv')
ans = f'{path}/sample_submission.csv'
submit = pd.read_csv(ans)

In [ ]:
label_mapping = {
    '가구수정': 'Furniture',
    '걸레받이수정': 'mopholder',
    '곰팡이': 'mold',
    '꼬임': 'twist',
    '녹오염': 'rust pollution',
    '들뜸': 'wrinkle',
    '면불량': 'cotton defect',
    '몰딩수정': 'molding',
    '반점': 'half spot',
    '석고수정': 'plaster',
    '오염': 'contamination',
    '오타공': 'black spot', 
    '울음': 'wailing',
    '이음부불량': 'bad joint',
    '창틀,문틀수정': 'window frame',
    '터짐': 'crack',
    '틈새과다': 'gap',
    '피스': 'piece',
    '훼손': 'damage'
}

In [ ]:
reversed_mapping = {value: key for key, value in label_mapping.items()}

In [ ]:
nums = len(label_mapping)

In [ ]:
num_label_mapping = {value: index for index, value in enumerate(label_mapping.values())}


In [ ]:
num_label_mapping

{'Furniture': 0,
 'mopholder': 1,
 'mold': 2,
 'twist': 3,
 'rust pollution': 4,
 'wrinkle': 5,
 'cotton defect': 6,
 'molding': 7,
 'half spot': 8,
 'plaster': 9,
 'contamination': 10,
 'black spot': 11,
 'wailing': 12,
 'bad joint': 13,
 'window frame': 14,
 'crack': 15,
 'gap': 16,
 'piece': 17,
 'damage': 18}

In [ ]:
folders = os.listdir(f'{path}/train')
folders

['Furniture',
 'mold',
 'rust pollution',
 'piece',
 'contamination',
 'damage',
 'twist',
 'mopholder',
 'wrinkle',
 'gap',
 'crack',
 'window frame',
 'bad joint',
 'wailing',
 'black spot',
 'plaster',
 'half spot',
 'molding',
 'cotton defect']

In [ ]:
label2sent = {
    '가구수정': 'the crack with drawer and wall',
    '걸레받이수정': 'gap or separation between the baseboard and wallpaper',
    '곰팡이': 'blue mold between the wallpaper surface or molding',
    '꼬임': 'wall paper is coming off at the corner',
    '녹오염': 'brwon spot on the wallpaper surface',
    '들뜸': 'the crack with drawer and wall',
    '면불량': 'bulge on the wall surface',
    '몰딩수정': 'crack between the ceiling and wall',
    '반점': 'red spot or blue spot on the wallpaper',
    '석고수정': 'the brown board is exposed',
    '오염': 'various colored lines or random shapes on the wallpaper surface',
    '오타공': 'black hole on the wall', 
    '울음': 'oval shapes bubbles on the wallpaper',
    '이음부불량': 'the gap between two wall',
    '창틀,문틀수정': 'the damage between door frame and window frame',
    '터짐': 'crack near electric outlet',
    '틈새과다': 'excessive gaps between wallpaper and molding',
    '피스': 'the crack under the wallpaper',
    '훼손': 'a hole or crack in the wallpaper'
}

In [ ]:
# for folder in folders:
#     en = label_mapping.get(folder)
#     os.rename(f'{path}/train/{folder}', f'{path}/train/{en}')

In [ ]:
# folders = os.listdir(f'{path}/train')
# folders

In [ ]:
img_glob = glob.glob(f'{path}/train/*/*')

In [ ]:
len(img_glob)

3400

In [ ]:
img_glob[:10]

['/content/gdrive/MyDrive/wall/train/Furniture/10.png',
 '/content/gdrive/MyDrive/wall/train/Furniture/7.png',
 '/content/gdrive/MyDrive/wall/train/Furniture/11.png',
 '/content/gdrive/MyDrive/wall/train/Furniture/2.png',
 '/content/gdrive/MyDrive/wall/train/Furniture/3.png',
 '/content/gdrive/MyDrive/wall/train/Furniture/8.png',
 '/content/gdrive/MyDrive/wall/train/Furniture/1.png',
 '/content/gdrive/MyDrive/wall/train/Furniture/9.png',
 '/content/gdrive/MyDrive/wall/train/Furniture/6.png',
 '/content/gdrive/MyDrive/wall/train/Furniture/0.png']

In [ ]:
img_glob[0].split('/')[-2]

'Furniture'

In [ ]:
train = pd.DataFrame(columns=['path', 'label'])
train['path'] = img_glob
train['label'] = train['path'].apply(lambda x: str(x).split('/')[-2])

In [ ]:
train

,path,label
0,/content/gdrive/MyDrive/wall/train/Furniture/1...,Furniture
1,/content/gdrive/MyDrive/wall/train/Furniture/7...,Furniture
2,/content/gdrive/MyDrive/wall/train/Furniture/1...,Furniture
3,/content/gdrive/MyDrive/wall/train/Furniture/2...,Furniture
4,/content/gdrive/MyDrive/wall/train/Furniture/3...,Furniture
...,...,...
3395,/content/gdrive/MyDrive/wall/train/cotton defe...,cotton defect
3396,/content/gdrive/MyDrive/wall/train/cotton defe...,cotton defect
3397,/content/gdrive/MyDrive/wall/train/cotton defe...,cotton defect
3398,/content/gdrive/MyDrive/wall/train/cotton defe...,cotton defect


In [ ]:
train['ko_label'] = train['label'].map(reversed_mapping)

In [ ]:
train['num_label'] = train['label'].map(num_label_mapping)

In [ ]:
train['sen_lebel'] = train['ko_label'].map(label2sent)

In [ ]:
train

,path,label,ko_label,num_label,sen_lebel
0,/content/gdrive/MyDrive/wall/train/Furniture/1...,Furniture,가구수정,0,the crack with drawer and wall
1,/content/gdrive/MyDrive/wall/train/Furniture/7...,Furniture,가구수정,0,the crack with drawer and wall
2,/content/gdrive/MyDrive/wall/train/Furniture/1...,Furniture,가구수정,0,the crack with drawer and wall
3,/content/gdrive/MyDrive/wall/train/Furniture/2...,Furniture,가구수정,0,the crack with drawer and wall
4,/content/gdrive/MyDrive/wall/train/Furniture/3...,Furniture,가구수정,0,the crack with drawer and wall
...,...,...,...,...,...
3395,/content/gdrive/MyDrive/wall/train/cotton defe...,cotton defect,면불량,6,bulge on the wall surface
3396,/content/gdrive/MyDrive/wall/train/cotton defe...,cotton defect,면불량,6,bulge on the wall surface
3397,/content/gdrive/MyDrive/wall/train/cotton defe...,cotton defect,면불량,6,bulge on the wall surface
3398,/content/gdrive/MyDrive/wall/train/cotton defe...,cotton defect,면불량,6,bulge on the wall surface


In [ ]:
test

,id,img_path
0,TEST_000,./test/000.png
1,TEST_001,./test/001.png
2,TEST_002,./test/002.png
3,TEST_003,./test/003.png
4,TEST_004,./test/004.png
...,...,...
787,TEST_787,./test/787.png
788,TEST_788,./test/788.png
789,TEST_789,./test/789.png
790,TEST_790,./test/790.png


In [ ]:
prefix = '/content/gdrive/MyDrive/wall/'
test['img_path'] = prefix+test['img_path'].str[2:]

In [ ]:
test.head()

,id,img_path
0,TEST_000,/content/gdrive/MyDrive/wall/test/000.png
1,TEST_001,/content/gdrive/MyDrive/wall/test/001.png
2,TEST_002,/content/gdrive/MyDrive/wall/test/002.png
3,TEST_003,/content/gdrive/MyDrive/wall/test/003.png
4,TEST_004,/content/gdrive/MyDrive/wall/test/004.png


### Down Sampling

In [ ]:
#너무적어서 의미가 없음

In [ ]:
# #df = train
# #mopholder
# class_counts = train['label'].value_counts()
# ds = 'mopholder'#목표 
# ds_cnt = class_counts['damage']#이거를 줄인다
# other_classes = train[train['label'] != ds].index
# downsampled_class = train[train['label'] == ds].index
# downsampled_data = train.loc[other_classes].sample(n=ds_cnt, random_state=69)
# downsampled_data = pd.concat([downsampled_data, train.loc[downsampled_class]])

# # train = downsampled_data
# # class_counts = train['label'].value_counts()
# # ds = 'mopholder'#목표 
# # ds_cnt2 = class_counts['contamination']#이거를 줄인다
# # other_classes = train[train['label'] != ds].index
# # downsampled_class = train[train['label'] == ds].index
# # downsampled_data = train.loc[other_classes].sample(n=ds_cnt2, random_state=69)
# # downsampled_data = pd.concat([downsampled_data, train.loc[downsampled_class]])

# train = downsampled_data
# train

In [ ]:
train.head()

,path,label,ko_label,num_label,sen_lebel
0,/content/gdrive/MyDrive/wall/train/Furniture/1...,Furniture,가구수정,0,the crack with drawer and wall
1,/content/gdrive/MyDrive/wall/train/Furniture/7...,Furniture,가구수정,0,the crack with drawer and wall
2,/content/gdrive/MyDrive/wall/train/Furniture/1...,Furniture,가구수정,0,the crack with drawer and wall
3,/content/gdrive/MyDrive/wall/train/Furniture/2...,Furniture,가구수정,0,the crack with drawer and wall
4,/content/gdrive/MyDrive/wall/train/Furniture/3...,Furniture,가구수정,0,the crack with drawer and wall


In [ ]:
train['ko_label'].value_counts()

ko_label
훼손                 1360
오염                  595
걸레받이수정        307
꼬임                  203
터짐                  162
곰팡이               145
오타공                142
몰딩수정            130
면불량               94
석고수정              57
들뜸                  54
피스                    51
창틀,문틀수정      27
울음                  22
이음부불량           17
녹오염                14
가구수정               12
틈새과다                5
반점                   3
Name: count, dtype: int64

In [ ]:
train, val, _, _ = train_test_split(train, train['num_label'], test_size = 0.25, stratify=train['num_label'], random_state=CFG['SEED'])

### Label-Encoding


In [ ]:
le = preprocessing.LabelEncoder()
train['num_label'] = le.fit_transform(train['num_label'])
val['num_label'] = le.transform(val['num_label'])

In [ ]:
le.classes_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        image = cv2.imread(img_path)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
        #for img+path in self.img_path_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        img = Image.open(img_path)
        #img = np.array(image)

        if self.transforms is not None:
            image = self.transforms(img)

        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
class TextDataset(Dataset):
    def __init__(self, img_path_list, label_list, text_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        self.text_list = text_list
        #for img+path in self.img_path_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        img = Image.open(img_path)
        #img = np.array(image)
        text = self.text_list[index]
        if self.transforms is not None:
            image = self.transforms(img)

        if self.label_list is not None:
            label = self.label_list[index]
            return image, label, text
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
#Resize, horizonflip, rotate, gaussianblur
# train_transform = A.Compose([
#     A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
#     A.RandomCrop(width=224, height=224),
#     A.RandomRotate90(),
#     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
#     ToTensorV2()
# ])
# test_transform = A.Compose([
#     A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
#     A.RandomCrop(width=224, height=224),
#     A.RandomRotate90(),
#     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
#     ToTensorV2()
# ])
train_transform = transforms.Compose([
    transforms.Resize(CFG['IMG_SIZE']),
    transforms.RandomResizedCrop(size=(224,224), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    transforms.TrivialAugmentWide(),
    transforms.RandomSolarize(threshold=192.0),
    transforms.RandomInvert(),
    transforms.TrivialAugmentWide(),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    #transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    
    transforms.RandomErasing(p=0.25),

])
test_transform = transforms.Compose([
    transforms.Resize(CFG['IMG_SIZE']),
    transforms.RandomResizedCrop(size=(224,224), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    transforms.TrivialAugmentWide(),
    transforms.RandomSolarize(threshold=192.0),
    transforms.RandomInvert(),
    transforms.TrivialAugmentWide(),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    #transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    
    transforms.RandomErasing(p=0.25)
    
])


In [ ]:
train_dataset = TestDataset(train['path'].values, train['num_label'].values, train_transform)
#train_dataset = TextDataset(train['path'].values, train['num_label'].values,train['sen_lebel'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = TestDataset(val['path'].values, val['num_label'].values, test_transform)
#val_dataset = TextDataset(val['path'].values, val['num_label'].values,val['sen_lebel'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
#image, label = train_dataset.__getitem__(0)

In [ ]:
#image

In [ ]:
#image, label = val_dataset.__getitem__(0)

In [ ]:
#label

### Model Define

In [ ]:
#7
class EfficientNetB2(nn.Module):
    def __init__(self, num_classes = len(le.classes_)):
        super(EfficientNetB2, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, num_classes)
        self.conv1 = self.create_layer(3, 32)
        self.conv2 = self.create_layer(32, 64)
        self.conv3 = self.create_layer(64, 128)
        self.conv4 = self.create_layer(128, 256)
        self.conv5 = self.create_layer(256, 512)
        self.conv6 = self.create_layer(512, 1024)
        self.conv7 = self.create_layer(1024, 2048)
    def create_layer(self, input, output):
        conv = nn.Conv2d(input, output, kernel_size=3, stride=2, padding=1)
        batch = nn.BatchNorm2d(output)
        relu = nn.ReLU()
        return nn.Sequential(conv, batch, relu)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
class Resnet50(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(Resnet50, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
class Resnet101(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(Resnet101, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b3(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
   
    #criterion =  FocalLoss().to(device)
    criterion =  nn.CrossEntropyLoss().to(device)
    #criterion =  FocalLoss().to(device)

    #FocalLoss
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    current_lr = optimizer.param_groups[0]['lr']
    model_path = f"{path}/model_{CFG['BATCH_SIZE']}_{current_lr}.pth"
    torch.save(best_model.state_dict(), model_path)
    return best_model

In [ ]:
#current_lr = optimizer.param_groups[0]['lr']

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

Imbalance 해결 방안1
Focal loss

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

In [ ]:
#model = BaseModel()
#model = EfficientNetB2()
# /content/gdrive/MyDrive/wall/model_32_0.0001.pth
model_path = f'/content/gdrive/MyDrive/wall/model_64_1.25e-05.pth'
#/content/gdrive/MyDrive/wall/model_64_6.25e-06.pth
load_Model = True
if load_Model:
    model = Resnet50()
    model.load_state_dict(torch.load(model_path))
    #model = torch.load(model_path)
else:
    model = Resnet50()

model.eval()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
infer_model = train(model, optimizer, train_loader, val_loader , scheduler, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.43174] Val Loss : [1.26358] Val Weighted F1 Score : [0.68436]


  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/14 [00:00<?, ?it/s]

Epoch [29], Train Loss : [0.20166] Val Loss : [0.92071] Val Weighted F1 Score : [0.74975]
Epoch 00029: reducing learning rate of group 0 to 7.8125e-07.


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

Epoch [30], Train Loss : [0.18052] Val Loss : [0.91653] Val Weighted F1 Score : [0.74444]


In [ ]:
current_lr = optimizer.param_groups[0]['lr']
current_lr

7.8125e-07

### Inference

In [ ]:
test = pd.read_csv(f'{path}/test.csv')

In [ ]:
prefix = '/content/gdrive/MyDrive/wall/'
test['img_path'] = prefix+test['img_path'].str[2:]
#/content/gdrive/MyDrive/wall/test/000.png

In [ ]:
test.head()

,id,img_path
0,TEST_000,/content/gdrive/MyDrive/wall/test/000.png
1,TEST_001,/content/gdrive/MyDrive/wall/test/001.png
2,TEST_002,/content/gdrive/MyDrive/wall/test/002.png
3,TEST_003,/content/gdrive/MyDrive/wall/test/003.png
4,TEST_004,/content/gdrive/MyDrive/wall/test/004.png


In [ ]:
#test_dataset = ImageDataset(test['img_path'].values, None, test_transform)
test_dataset = TestDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            #label = label.to(device)
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
ubmit = pd.read_csv(f'{path}/sample_submission.csv')

In [ ]:
ubmit

,id,label
0,TEST_000,훼손
1,TEST_001,훼손
2,TEST_002,훼손
3,TEST_003,훼손
4,TEST_004,훼손
...,...,...
787,TEST_787,훼손
788,TEST_788,훼손
789,TEST_789,훼손
790,TEST_790,훼손


In [ ]:
submit['label'] = preds

In [ ]:
label_mapping_reverse = {v: k for k, v in label_mapping.items()}
#label_mapping_reverse = {label_mapping_reverse[k]: k for k in label_mapping_reverse}

In [ ]:
num_label_mapping2 = {value: index for index, value in enumerate(label_mapping_reverse.values())}
label_mapping_reverse2 = {v: k for k, v in num_label_mapping2.items()}


In [ ]:
label_mapping_reverse2

{0: '가구수정',
 1: '걸레받이수정',
 2: '곰팡이',
 3: '꼬임',
 4: '녹오염',
 5: '들뜸',
 6: '면불량',
 7: '몰딩수정',
 8: '반점',
 9: '석고수정',
 10: '오염',
 11: '오타공',
 12: '울음',
 13: '이음부불량',
 14: '창틀,문틀수정',
 15: '터짐',
 16: '틈새과다',
 17: '피스',
 18: '훼손'}

In [ ]:
submit['label'] = submit['label'].map(label_mapping_reverse2)

In [ ]:
answer_value = submit['label'].tolist()

In [ ]:
import unicodedata


label_data = [unicodedata.normalize('NFC', one_ans_val) for one_ans_val in answer_value]

submit["label"] = label_data

print("\n label transformation done.")


 label transformation done.


In [ ]:
submit

,id,label
0,TEST_000,훼손
1,TEST_001,훼손
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염
...,...,...
787,TEST_787,"창틀,문틀수정"
788,TEST_788,훼손
789,TEST_789,오염
790,TEST_790,오염


In [ ]:
submit.to_csv('./baseline_submit.csv', index=False, encoding = 'utf-8-sig')